In [1]:
from easydict import EasyDict as edict
import json, os
import torch

import pandas as pd
import numpy as np

from scipy.special import expit
import sklearn.metrics as skm

import matplotlib.pyplot as plt

#### Chexpert Code

In [2]:
import sys
sys.path.append('/home/users/jschott/Chexpert/')
from model.classifier import Classifier
from data.dataset import ImageDataset 

import glob
glob.glob('/work/projects/covid19_dv/jessica_ba/*/test_custom_predictions.csv')

['/work/projects/covid19_dv/jessica_ba/results_1-5_female_70k_3/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_1-5_female_70k_4/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_1-5_female_70k_1/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_male_2_70k/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_3_male_70k/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_1_male_70k/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_female/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_70k/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_1-5_female_70k_2/test_custom_predictions.csv',
 '/work/projects/covid19_dv/jessica_ba/results_1-5_female_70k_5/test_custom_predictions.csv',
 '/work/projects/covid19

## Load predictions of different models

In [9]:
validation_truth = '/work/projects/covid19_dv/jessica_ba/config/valid_exp1.csv'

# alle Predictions die geladen werden sollen, mit namen unter dem Sie verwendet werden sollen
prediction_files = {
    'full_population': '/work/projects/covid19_dv/jessica_ba/results/test_custom_predictions.csv',
    'only_female': '/work/projects/covid19_dv/jessica_ba/results_female/test_custom_predictions.csv',
    'population_70k': '/work/projects/covid19_dv/jessica_ba/results_70k/test_custom_predictions.csv',
    'only_female_70k': '/work/projects/covid19_dv/jessica_ba/results_female_70k/test_custom_predictions.csv',
    'only_male_70k': '/work/projects/covid19_dv/jessica_ba/results_1_male_70k/test_custom_predictions.csv',
    'only_male_70k_2': '/work/projects/covid19_dv/jessica_ba/results_male_2_70k/test_custom_predictions.csv',
    'only_male_70k_3': '/work/projects/covid19_dv/jessica_ba/results_3_male_70k/test_custom_predictions.csv',
    'only_male_70k_4'
}

In [10]:
truth_df = pd.read_csv('/work/projects/covid19_dv/jessica_ba/config/valid_exp1.csv')

In [11]:
probabilities = {}
for exp_name, file in prediction_files.items(): 
    probabilities[exp_name] = pd.read_csv(file)

In [12]:
truth_df.columns

Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices', 'PatientId'],
      dtype='object')

In [13]:
col = 'Edema' #'Consolidation'# #'Cardiomegaly'
#population_selection = truth_df.Sex.isin(['Female', 'Male'])
#population_selection = truth_df.Sex.isin(['Male'])
population_selection = truth_df.Sex.isin(['Female'])

for exp_name in prediction_files: 

    targets = truth_df[col].fillna(0)    # Spalte mit 0en Auffüllen
    not_ignore = targets >= 0            # ignoriere alle Werte kleiner 0
    #print(f'{(~not_ignore).sum()} images neglected due to -1') # formatiert den String (die Variable in den geschweiften Klammern)
    
    auc = skm.roc_auc_score( # berechne, wie gut das Modell vorhersagt für die jeweilige Krankheit mit den Werten >= 0
        targets[not_ignore & population_selection], 
        probabilities[exp_name][col][not_ignore & population_selection]) 
    
    lloss = skm.log_loss(
        targets[not_ignore & population_selection], 
        probabilities[exp_name][col][not_ignore & population_selection]
    )

    print(f'{exp_name} {col} AUC: {auc:.3f}, log loss: {lloss:.3f}') 

full_population Edema AUC: 0.855, log loss: 0.567
only_female Edema AUC: 0.832, log loss: 0.581
population_70k Edema AUC: 0.852, log loss: 0.584
only_female_70k Edema AUC: 0.812, log loss: 0.616
only_male_70k Edema AUC: 0.835, log loss: 0.572
only_male_70k_2 Edema AUC: 0.840, log loss: 0.592
only_male_70k_3 Edema AUC: 0.848, log loss: 0.576
